## Dự đoán giá đất theo dữ liệu csv

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Đọc dữ liệu từ file CSV
df = pd.read_csv("giadat.csv")

# Kiểm tra dữ liệu
print(df.head())

# Xử lý dữ liệu: Chuyển đổi các cột dạng chuỗi thành số bằng Label Encoding
label_encoders = {}
for col in ["Khu vực", "Loại đất"]:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

# Chia dữ liệu thành đầu vào (X) và đầu ra (y)
X = df[["Khu vực", "Loại đất", "Vị trí"]]
y = df["Giá đất"]

# Chia tập dữ liệu thành tập huấn luyện (80%) và tập kiểm tra (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Huấn luyện mô hình Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse:.2f}")
print(f"R²: {r2:.2f}")

# Hàm dự đoán giá đất
def predict_price(khu_vuc, loai_dat, vi_tri):
    # Chuyển đổi dữ liệu đầu vào theo Label Encoding
    if khu_vuc in label_encoders["Khu vực"].classes_:
        khu_vuc = label_encoders["Khu vực"].transform([khu_vuc])[0]
    else:
        print("Lỗi: Khu vực không tồn tại trong dữ liệu!")
        return None

    if loai_dat in label_encoders["Loại đất"].classes_:
        loai_dat = label_encoders["Loại đất"].transform([loai_dat])[0]
    else:
        print("Lỗi: Loại đất không tồn tại trong dữ liệu!")
        return None

    # Chuẩn hóa đầu vào
    input_data = np.array([[khu_vuc, loai_dat, vi_tri]])
    input_data = scaler.transform(input_data)

    # Dự đoán giá
    predicted_price = model.predict(input_data)[0]
    return round(predicted_price, 0)  # Làm tròn giá trị dự đoán

# Ví dụ sử dụng: Dự đoán giá đất tại "Bà Rịa", "Đất ở", vị trí 2
predicted_price = predict_price("Bà Rịa", "Đất ở", 2)
if predicted_price:
    print(f"Dự đoán giá đất: {predicted_price:,.0f} VND/m²")

     Khu vực         Loại đất  Vị trí   Giá đất
0   Vũng Tàu            Đất ở       1  27400000
1     Bà Rịa  Đất nông nghiệp       2   9100000
2     Phú Mỹ   Đất thương mại       3   5850000
3  Long Điền            Đất ở       4   2500000
4   Châu Đức         Đất rừng       5   1600000
MSE: 312639988990600.00
R²: -2.73
Dự đoán giá đất: 3,183,480 VND/m²


/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
